<a href="https://colab.research.google.com/github/AllysonAbreu/dw_contas_publicas_2022/blob/master/analise_regressiva_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [11]:
df_receitas = pd.read_csv('dados_receitas.csv', sep=';')
df_receitas

,COD_RECEITA,DATA,COD_CONTRIBUINTE,VALOR_ARRECADADO,FONTE_DADOS
0,1722010101,2013-01-03,886cd0eabf5a18,185560.68,prefeitura municipal
1,1722010101,2013-01-04,886cd0eabf5a18,8081.80,prefeitura municipal
2,1722010101,2013-01-07,886cd0eabf5a18,4231.39,prefeitura municipal
3,1722010101,2013-01-08,886cd0eabf5a18,10536.53,prefeitura municipal
4,1722010101,2013-01-15,886cd0eabf5a18,103118.27,prefeitura municipal
...,...,...,...,...,...
102288,191101011,2022-01-19,e6da541375c18b,4374.24,sctrans
102289,112101011,2022-01-18,e6da541375c18b,228.00,sctrans
102290,191101011,2022-01-18,e6da541375c18b,3758.69,sctrans
102291,112101011,2022-01-17,e6da541375c18b,4297.00,sctrans


In [12]:
df_receitas_copy = df_receitas[['COD_RECEITA','DATA','COD_CONTRIBUINTE','VALOR_ARRECADADO']].copy()
df_receitas_copy.rename(columns={'COD_RECEITA':'cod_receita','DATA':'data_fato','COD_CONTRIBUINTE':'contribuinte_receita','VALOR_ARRECADADO':'valor'}, inplace=True)
df_receitas_copy['data_fato'] = pd.to_datetime(df_receitas_copy['data_fato'], format='%Y-%m-%d')
df_receitas_copy['ano'] = df_receitas_copy['data_fato'].dt.year
df_receitas_copy['mes'] = df_receitas_copy['data_fato'].dt.month
df_receitas_copy['dia'] = df_receitas_copy['data_fato'].dt.day
df_receitas_copy['mes_ano'] = df_receitas_copy['data_fato'].dt.strftime('%m-%Y')
df_receitas_copy

,cod_receita,data_fato,contribuinte_receita,valor,ano,mes,dia,mes_ano
0,1722010101,2013-01-03,886cd0eabf5a18,185560.68,2013,1,3,01-2013
1,1722010101,2013-01-04,886cd0eabf5a18,8081.80,2013,1,4,01-2013
2,1722010101,2013-01-07,886cd0eabf5a18,4231.39,2013,1,7,01-2013
3,1722010101,2013-01-08,886cd0eabf5a18,10536.53,2013,1,8,01-2013
4,1722010101,2013-01-15,886cd0eabf5a18,103118.27,2013,1,15,01-2013
...,...,...,...,...,...,...,...,...
102288,191101011,2022-01-19,e6da541375c18b,4374.24,2022,1,19,01-2022
102289,112101011,2022-01-18,e6da541375c18b,228.00,2022,1,18,01-2022
102290,191101011,2022-01-18,e6da541375c18b,3758.69,2022,1,18,01-2022
102291,112101011,2022-01-17,e6da541375c18b,4297.00,2022,1,17,01-2022


In [13]:
# Contagem de valores ausentes por coluna
print(df_receitas_copy.isnull().sum())

# Remoção dos valores ausentes
df_receitas_copy.dropna(inplace=True)

cod_receita             0
data_fato               0
contribuinte_receita    0
valor                   0
ano                     0
mes                     0
dia                     0
mes_ano                 0
dtype: int64


<p>Regressão linear simples: considerando apenas uma variável independente, como por exemplo, a "data_fato", e uma variável dependente, como o "valor". Nesse caso, pode-se avaliar como o valor da receita varia ao longo do tempo.</p>

In [15]:
# Cria uma coluna com valores numéricos para a data
df_receitas_copy['data_numerica'] = pd.to_datetime(df_receitas_copy['data_fato']).astype(int)

# Cria um objeto do tipo LinearRegression
regressor = LinearRegression()

# Define as variáveis independentes e dependentes
X = df_receitas_copy[['data_numerica']]
y = df_receitas_copy['valor']

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treina o modelo de regressão
regressor.fit(X_train, y_train)

# Faz a previsão para um novo valor de data
data_nova = pd.Timestamp('2013-04-22')
data_nova_numerica = data_nova.to_pydatetime().timestamp()
valor_previsto = regressor.predict([[data_nova_numerica]])

# Imprime o valor previsto para a nova data
print(f'Para a data {data_nova}, o valor previsto é de R${valor_previsto[0]:.2f}.')

# Avaliar o modelo usando os dados de teste
score = regressor.score(X_test, y_test)
print(f"A acurácia do modelo é {score}")

Para a data 2013-04-22 00:00:00, o valor previsto é de R$-59939.10.
A acurácia do modelo é 0.0027920019767447846


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
